In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from gears import PertData, GEARS
import numpy as np
import pandas as pd
from tqdm import tqdm
import scanpy as sc
import anndata
from scipy.sparse import csr_matrix

In [4]:
data_dir = '/home/vinas/sc_perturbation_benchmark/data/xu_kinetics_2024'

#### Download data

Download data from:
```
https://ftp.ncbi.nlm.nih.gov/geo/series/GSE218nnn/GSE218566/suppl/GSE218566_RAW.tar
```

#### Process data

In [20]:
adata_ = anndata.read_mtx(f'{data_dir}/GSM6752591_on_target_whole_tx_count_matrix.mtx.gz')
obs = pd.read_csv(f'{data_dir}/GSM6752591_on_target_cell_metadata.csv.gz', sep=',', compression='gzip')
var = pd.read_csv(f'{data_dir}/GSM6752591_on_target_whole_tx.Genes.tsv.gz', sep=',', compression='gzip', header=None)
var.columns = ['gene_name']
adata = anndata.AnnData(adata_.X.T, obs=obs, var=var)

In [28]:
genes = adata.obs['target_genes'].unique().tolist()

for i, gene in enumerate(genes):
    if gene not in adata.var['gene_name'].values:
        print(gene)

NO-TARGET


In [30]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=5000, subset=False)

In [31]:
adata.obs['gene'] = adata.obs['target_genes']
hvg_flag_ = adata.var['highly_variable'].values
gene_flag_ = adata.var['gene_name'].isin(adata.obs['gene'].values).values
select_flag_ = np.logical_or(hvg_flag_, gene_flag_)
condition_flag_ = adata.obs['gene'].isin(adata.var['gene_name'].values.tolist()+['NO-TARGET']).values
adata_subset = adata[condition_flag_, select_flag_]

In [33]:
# Prepare data in GEARS format
adata_subset.obs['condition'] = [i+'+ctrl' for i in adata_subset.obs['gene'].values]
adata_subset.obs['condition'] = adata_subset.obs['condition'].replace({'NO-TARGET+ctrl': 'ctrl'})
adata_subset.obs['cell_type'] = 'HEK293'
adata_subset.obs['condition'].value_counts()

condition
ctrl             2758
SMARCD1+ctrl     2158
SLC25A1+ctrl     1880
XPO5+ctrl        1583
LDHA+ctrl        1437
                 ... 
RBM8A+ctrl         16
RUVBL1+ctrl        10
RPL10+ctrl          9
HEATR1+ctrl         9
ATP6V1B2+ctrl       3
Name: count, Length: 204, dtype: int64

In [34]:
out_dir = data_dir
pert_data = PertData(out_dir)
pert_data.new_data_process(dataset_name = 'HEK293', adata = adata_subset)

Downloading...
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9.46M/9.46M [00:01<00:00, 8.48MiB/s]
Downloading...
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 559k/559k [00:00<00:00, 1.41MiB/s]
Creating pyg object for each cell in the data...
Creating dataset file...
 39%|████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                                                                                                                  | 80/204 [25

SLC28A1+ctrl


 46%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                                                                                                 | 93/204 [28:05<18:35, 10.05s/it]

SLC29A3+ctrl


 57%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                                                  | 116/204 [34:11<22:00, 15.00s/it]

...

 58%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                                                                | 118/204 [34:47<22:47, 15.90s/it]

SLC28A3+ctrl


 64%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                                               | 131/204 [37:32<13:45, 11.31s/it]

ATP12A+ctrl


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 204/204 [44:24<00:00, 13.06s/it]
Done!
Saving new dataset pyg object at /home/vinas/sc_perturbation_benchmark/data/xu_kinetics_2024/hek293/data_pyg/cell_graphs.pkl
Done!
